# Range walk in a spherical earth geometry
from the appendix of the thesis *"Multi-channel Azimuth Processing for High-Resolution Wide-Swath SAR Imaging"* by N. Gerbert:
\begin{equation}
    R(t) = \sqrt{(R_E + h)^2 + R_E^2 -2(R_E+h)R_E \cos(\theta_e) \cos\left(\dfrac{v_s}{R_E + h}t\right)  }
\end{equation}
\begin{equation}
\theta_e = \dfrac{R_g}{R_E}
\end{equation}
$R_g$ point ground range, $R_E$ Earth radius
$\theta_e$ can also be written as function of the slant (closest approach) range and/or the incidence angle

lets find the derivatives and taylor expansion for a fixed $\theta_e$ (ground range point)
to simplify:
$ A = (R_E + h)^2 + R_E^2 = \text{constant}$\
$ B = 2(R_E+h)R_E \cos(\theta_e)= \text{constant}$\
$\nu = \dfrac{v_s}{R_E + h} = \text{constant}$\
then
$R(t) = \sqrt{A-B\cos(\nu t)}$

In [37]:
import sympy as sp

a, b, v, t = sp.symbols(('A', 'B', '\\nu', 't'))
Re, h, theta_e, vs = sp.symbols(('R_e', 'h', '\\theta_e', 'v_s'))
A = (Re + h) ** 2 + Re ** 2
B = 2 * (Re + h) * Re * sp.cos(theta_e)
Nu = vs / (Re + h)
range = sp.sqrt(a - b * sp.cos(v * t))
range.series(t, 0, n=5)

sqrt(A - B) + t**4*sqrt(A - B)*(-B**2*\nu**4/(32*(A - B)**2) - B*\nu**4/(48*(A - B))) + B*\nu**2*t**2/(4*sqrt(A - B)) + O(t**5)

In [38]:
eq = sp.sqrt(a - b)
eq

sqrt(A - B)

In [39]:
eq.subs([(a, A), (b, B)])

sqrt(R_e**2 - R_e*(2*R_e + 2*h)*cos(\theta_e) + (R_e + h)**2)

with the first derivative being

In [40]:
print("R\'(t)")
r1 = sp.diff(range, t)
r1

R'(t)


B*\nu*sin(\nu*t)/(2*sqrt(A - B*cos(\nu*t)))

In [41]:
print('=')

r1a = r1.subs([(a, A), (b, B)])
r1a.simplify()

=


R_e*\nu*(R_e + h)*sin(\nu*t)*cos(\theta_e)/sqrt(R_e**2 - 2*R_e*(R_e + h)*cos(\theta_e)*cos(\nu*t) + (R_e + h)**2)

and the second derivative

In [42]:
print("R\'\'(t)")
r2 = (sp.diff(range, t, 2))
r2 = r2.simplify()
r2

R''(t)


-B*\nu**2*(-2*A*cos(\nu*t) + B*cos(\nu*t)**2 + B)/(4*(A - B*cos(\nu*t))**(3/2))

In [54]:
print(latex(1 / r2))

- \frac{4 \left(A - B \cos{\left(\nu t \right)}\right)^{\frac{3}{2}}}{B \nu^{2} \left(- 2 A \cos{\left(\nu t \right)} + B \cos^{2}{\left(\nu t \right)} + B\right)}


In [44]:
pippo = r2.subs([(a, A), (b, B)])
pippo.simplify()

-R_e*\nu**2*(R_e + h)*(R_e*(R_e + h)*cos(\theta_e)*cos(\nu*t)**2 + R_e*(R_e + h)*cos(\theta_e) - (R_e**2 + (R_e + h)**2)*cos(\nu*t))*cos(\theta_e)/(R_e**2 - 2*R_e*(R_e + h)*cos(\theta_e)*cos(\nu*t) + (R_e + h)**2)**(3/2)

# Azimuth axis definition
The azimuth distance is defined here as the distance from a reference point on a circle with constant $\theta_e$
i.e. choosing the appropriate time reference
\begin{equation}
\text{Az} = R_E\dfrac{v_s}{R_E + h}t\cos(\Theta_E)
\end{equation}
linearly dependent from $t$

# Range Doppler signal stationary phase approximation
The received radar slow time signal is in the form:
\begin{equation}
    u(t) = a(t) \exp\left[ j\Theta(t)\right]
\end{equation}
where $\Theta(t)$ is the phase shift associated to the true time delay of a single point on earth.
Therefore
\begin{equation}
    \Theta(t) = - 4 \pi \dfrac{R(t)}{\lambda}
\end{equation}
with $\lambda$ = signal wavelength

The (Doppler) spectrum of u(t) is then, using POSP:
\begin{equation}
    U(f) \approx \sqrt{2 \pi}\ \dfrac{a(t_k)}{\sqrt{\left| \Theta''(t_k)\right|}} \exp\left\{ j \left[ -2\pi f\ t_k + \Theta(t_k) + \dfrac{\pi}{4}\right]\right\}
\end{equation}
Note $t_k$ is a function of frequency\
I.e. $t_k$ is the **stationary phase point** relating time to frequency, defined as:
\begin{equation}
    2 \pi f = \Theta'(t_k)
\end{equation}

## Slow Time to Doppler frequency coordinate conversion
\begin{equation}
    f_k = \dfrac{\Theta'(t)}{2 \pi} = \dfrac{- 4 \pi R'(t)}{2 \pi \lambda} = \dfrac{- 2 R'(t)}{\lambda}
\end{equation}

In [45]:
from sympy import *

ri, lam, f = symbols(("R'", '\lambda', 'f'))

freq = - 2 * ri / lam
freq = freq.subs(ri, r1)
freq


-B*\nu*sin(\nu*t)/(\lambda*sqrt(A - B*cos(\nu*t)))

In [46]:
latex(freq ** 2)

'\\frac{B^{2} \\nu^{2} \\sin^{2}{\\left(\\nu t \\right)}}{\\lambda^{2} \\left(A - B \\cos{\\left(\\nu t \\right)}\\right)}'

## Doppler Frequency to Slow time coordinate conversion
we just need to solve the above for t
\begin{equation}
 f^2 = \frac{B^{2} \nu^{2} \sin^{2}{\left(\nu t \right)}}{\lambda^{2} \left(A - B \cos{\left(\nu t \right)}\right)}
\end{equation}
using only the cosine
\begin{equation}
 f^2 = \frac{B^{2} \nu^{2} \left(1-\cos^{2}{\left(\nu t \right)}\right)}{\lambda^{2} \left(A - B \cos{\left(\nu t \right)}\right)}
\end{equation}

substituting $cos(\nu t)$ with x
\begin{equation}
 f^2 = \frac{B^{2} \nu^{2} \left(1-x^2\right)}{\lambda^{2} \left(A - B x\right)}
\end{equation}


In [47]:
x = symbols('x')
equation = b ** 2 * v ** 2 * (1 - x ** 2) / (lam ** 2 * (a - b * x)) - f ** 2
equation

B**2*\nu**2*(1 - x**2)/(\lambda**2*(A - B*x)) - f**2

In [48]:
nn = solve(equation, x)
nn

[(\lambda**2*f**2 - sqrt(-4*A*\lambda**2*\nu**2*f**2 + 4*B**2*\nu**4 + \lambda**4*f**4))/(2*B*\nu**2),
 (\lambda**2*f**2 + sqrt(-4*A*\lambda**2*\nu**2*f**2 + 4*B**2*\nu**4 + \lambda**4*f**4))/(2*B*\nu**2)]

In [49]:
nn[0].simplify()

(\lambda**2*f**2 - sqrt(-4*A*\lambda**2*\nu**2*f**2 + 4*B**2*\nu**4 + \lambda**4*f**4))/(2*B*\nu**2)

In [50]:
print(latex(nn[1].simplify()))

\frac{\lambda^{2} f^{2} + \sqrt{- 4 A \lambda^{2} \nu^{2} f^{2} + 4 B^{2} \nu^{4} + \lambda^{4} f^{4}}}{2 B \nu^{2}}


tHIS OnE has physical meaning

Then assuming the choice of the root is symmetrical with respect of the cosine:
\begin{equation}
    \cos(\nu t) = \frac{\lambda^{2} f^{2} + \sqrt{- 4 A \lambda^{2} \nu^{2} f^{2} + 4 B^{2} \nu^{4} + \lambda^{4} f^{4}}}{2 B \nu^{2}}
\end{equation}

considering that we are only interested in $|\nu t| < \pi/2$ (horizon) we can write:
\begin{equation}
    t_k =\pm \dfrac{1}{\nu}\ \arccos{\left(\frac{\lambda^{2} f^{2} + \sqrt{- 4 A \lambda^{2} \nu^{2} f^{2} + 4 B^{2} \nu^{4} + \lambda^{4} f^{4}}}{2 B \nu^{2}}\right)}
\end{equation}
Remembering that negative t have positive doppler shifts and vice versa\
t can now be related to a point in azimuth

## Stationary Phase Spectrum amplitude
\begin{equation}
|U(f)| \approx \sqrt{2 \pi}\ \dfrac{a(t_k)}{\sqrt{\left| \Theta''(t_k)\right|}}
\end{equation}

the stationary point scaling factor in doppler is then:

\begin{equation}
    S_f = \dfrac{\sqrt{2 \pi}}{\sqrt{\left| \Theta''(t_k)\right|}} = \sqrt{\dfrac{\lambda}{2\ |R''(t_k)|}} = \sqrt{\lambda}
    \sqrt{ \left| - \frac{2 \left(A - B \cos{\left(\nu t \right)}\right)^{\frac{3}{2}}}{B \nu^{2} \left(- 2 A \cos{\left(\nu t \right)} + B \cos^{2}{\left(\nu t \right)} + B\right)} \right|}
\end{equation}


In [51]:
Sf = sqrt(lam / (2 * abs(r2)))
Sf

sqrt(2)*sqrt(\lambda/Abs(B*\nu**2*(-2*A*cos(\nu*t) + B*cos(\nu*t)**2 + B)))*sqrt(Abs((A - B*cos(\nu*t))**(3/2)))

In [5]:
from sympy import *

# r0 from looking angle
r0, re, h, costhetal = symbols(('R_0', 'R_e', 'h', '\\cos{\\theta_L}'))

In [7]:
eq = r0 ** 2 - r0 * costhetal * 2 * (re + h) - re ** 2 + (re + h) ** 2
eq

R_0**2 - 2*R_0*\cos{\theta_L}*(R_e + h) - R_e**2 + (R_e + h)**2

In [35]:
aa = solve(eq, r0)[0].simplify()
print(aa)

\cos{\theta_L}*(R_e + h) - sqrt(R_e**2*\cos{\theta_L}**2 + 2*R_e*\cos{\theta_L}**2*h - 2*R_e*h + \cos{\theta_L}**2*h**2 - h**2)


In [26]:
bb= solve(eq, r0)[1].simplify()
bb

\cos{\theta_L}*(R_e + h) + sqrt(R_e**2*\cos{\theta_L}**2 + 2*R_e*\cos{\theta_L}**2*h - 2*R_e*h + \cos{\theta_L}**2*h**2 - h**2)

In [33]:
aa.subs([(costhetal, 1), (re,6371e3), (h, 500e3)]).evalf()

500000.000000000

In [34]:
bb.subs([(costhetal, 1), (re, 6371e3), (h, 500e3)]).evalf()

13242000.0000000